In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Connection to the Postgres database and Retrive data from the tables

In [2]:
#Read Data from the PostgressSql
conn = pg.connect(user = "postgres",
                                  password = "akshaj10",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "movie_data")
sql1 = "Select * From sherlock_ratings;"
sql2 = "Select * from award;"
movie_df = pd.read_sql_query(sql1, conn)
award_df = pd.read_sql_query(sql2, conn)
conn = None


In [3]:
#Load Data into Pandas Data frame and Display the dataframe.
movie_df.shape

(6052, 42)

In [4]:
#Load Data into Pandas Data frame and Display the dataframe.
award_df.shape

(10395, 8)

In [5]:
#Merge the two dataframe
combined_df = pd.merge(movie_df, award_df, left_on='title', right_on='film' )
combined_df.shape

(2351, 50)

In [6]:
columns = ['title', 'original_title', 'tagline','runtime',
       'budget', 'revenue', 'release_date', 'popularity', 'vote_average',
       'vote_count', 'genres', 'original_language', 'overview',
       'spoken_languages', 'country', 'production_companies',
       'production_countries', 'distributor', 'producers', 'director',
       'starring', 'cinematography', 'editors', 'writers', 'composers',
       'based_on', 'rating_0.5', 'rating_1.0', 'rating_1.5', 'rating_2.0',
       'rating_2.5', 'rating_3.0', 'rating_3.5', 'rating_4.0', 'rating_4.5',
       'rating_5.0', 'year_film', 'year_ceremony', 'ceremony',
       'category', 'name', 'film', 'winner']

target = ['winner']

In [7]:
# Filter all rows for US as the country
 
movie_us_df = combined_df[combined_df['country'] == 'United States'] 
movie_us_df.shape

(1720, 50)

In [8]:
# Find null values
for column in movie_us_df.columns:
    print(f"Column{column} has {movie_us_df[column].isnull().sum()} null values")

Columnindex_x has 0 null values
Columnimdb_id has 0 null values
Columnkaggle_id has 0 null values
Columntitle has 0 null values
Columnoriginal_title has 0 null values
Columntagline has 135 null values
Columnbelongs_to_collection has 1487 null values
Columnwikipedia_url has 0 null values
Columnimdb_link has 0 null values
Columnruntime has 0 null values
Columnbudget has 86 null values
Columnrevenue has 50 null values
Columnrelease_date has 0 null values
Columnpopularity has 0 null values
Columnvote_average has 0 null values
Columnvote_count has 0 null values
Columngenres has 0 null values
Columnoriginal_language has 0 null values
Columnoverview has 1 null values
Columnspoken_languages has 0 null values
Columncountry has 0 null values
Columnproduction_companies has 0 null values
Columnproduction_countries has 0 null values
Columndistributor has 3 null values
Columnproducers has 7 null values
Columndirector has 0 null values
Columnstarring has 16 null values
Columncinematography has 103 nu

#### Data Clean Up ,Preprocessing and Filtering

In [9]:
columns1 = ['title', 'original_title','runtime',
       'budget', 'revenue', 'popularity', 'vote_average',
       'vote_count', 'genres', 'original_language', 
       'spoken_languages', 'country', 'production_companies',
       'production_countries', 'distributor', 'producers', 'director',
       'starring', 'cinematography', 'editors', 'writers', 'composers',
       'rating_0.5', 'rating_1.0', 'rating_1.5', 'rating_2.0',
       'rating_2.5', 'rating_3.0', 'rating_3.5', 'rating_4.0', 'rating_4.5',
       'rating_5.0', 'year_film', 'year_ceremony', 'ceremony',
       'category', 'name', 'winner']

In [10]:
movie_us_df = movie_us_df.loc[:, columns1].copy()
movie_us_df.shape

(1720, 38)

In [11]:
movie_us_df.set_index('original_title')
movie_us_df

,title,original_title,runtime,budget,revenue,popularity,vote_average,vote_count,genres,original_language,...,rating_3.5,rating_4.0,rating_4.5,rating_5.0,year_film,year_ceremony,ceremony,category,name,winner
0,Alice,Alice,102.0,12000000.0,7331647.0,7.196816,6.3,57.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,...,0.0,0.0,0.0,0.0,1990,1991,63,WRITING (Screenplay Written Directly for the S...,Woody Allen,False
2,Avalon,Avalon,126.0,NaN,15700000.0,1.949265,6.0,19.0,"[{'id': 18, 'name': 'Drama'}]",en,...,2236.0,5329.0,1147.0,1940.0,1990,1991,63,CINEMATOGRAPHY,Allen Daviau,False
3,Avalon,Avalon,126.0,NaN,15700000.0,1.949265,6.0,19.0,"[{'id': 18, 'name': 'Drama'}]",en,...,2236.0,5329.0,1147.0,1940.0,1990,1991,63,COSTUME DESIGN,Gloria Gresham,False
4,Avalon,Avalon,126.0,NaN,15700000.0,1.949265,6.0,19.0,"[{'id': 18, 'name': 'Drama'}]",en,...,2236.0,5329.0,1147.0,1940.0,1990,1991,63,MUSIC (Original Score),Randy Newman,False
5,Avalon,Avalon,126.0,NaN,15700000.0,1.949265,6.0,19.0,"[{'id': 18, 'name': 'Drama'}]",en,...,2236.0,5329.0,1147.0,1940.0,1990,1991,63,WRITING (Screenplay Written Directly for the S...,Barry Levinson,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,The Boss Baby,The Boss Baby,97.0,125000000.0,498814908.0,13.392824,6.1,2336.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,...,0.0,0.0,0.0,0.0,2017,2018,90,ANIMATED FEATURE FILM,Tom McGrath and Ramsey Naito,False
2326,The Promise,The Promise,130.0,90000000.0,12400000.0,9.555114,7.3,69.0,"[{'id': 36, 'name': 'History'}, {'id': 10749, ...",en,...,0.0,0.0,0.0,0.0,1979,1980,52,MUSIC (Original Song),Music by David Shire; Lyric by Alan Bergman an...,False
2327,Guardians of the Galaxy Vol. 2,Guardians of the Galaxy Vol. 2,137.0,200000000.0,863416141.0,185.330992,7.6,4858.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,...,0.0,0.0,0.0,0.0,2017,2018,90,VISUAL EFFECTS,"Christopher Townsend, Guy Williams, Jonathan F...",False
2332,The Big Sick,The Big Sick,120.0,5000000.0,52620184.0,23.424794,7.7,249.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,...,0.0,0.0,0.0,0.0,2017,2018,90,WRITING (Original Screenplay),Written by Emily V. Gordon & Kumail Nanjiani,False


In [12]:
movie_us_df["winner"] = movie_us_df["winner"].replace({True: 1, False: 0})
movie_us_df.head(2)

,title,original_title,runtime,budget,revenue,popularity,vote_average,vote_count,genres,original_language,...,rating_3.5,rating_4.0,rating_4.5,rating_5.0,year_film,year_ceremony,ceremony,category,name,winner
0,Alice,Alice,102.0,12000000.0,7331647.0,7.196816,6.3,57.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,...,0.0,0.0,0.0,0.0,1990,1991,63,WRITING (Screenplay Written Directly for the S...,Woody Allen,0
2,Avalon,Avalon,126.0,NaN,15700000.0,1.949265,6.0,19.0,"[{'id': 18, 'name': 'Drama'}]",en,...,2236.0,5329.0,1147.0,1940.0,1990,1991,63,CINEMATOGRAPHY,Allen Daviau,0


In [13]:
# Generate our categorical variable list for movie dataset
movie_cat = movie_us_df.dtypes[movie_us_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movie_us_df[movie_cat].nunique()

title                    616
original_title           616
genres                   281
original_language          3
spoken_languages         104
country                    1
production_companies     512
production_countries      31
distributor              157
producers                573
director                 406
starring                 612
cinematography           296
editors                  429
writers                  539
composers                249
category                  77
name                    1357
dtype: int64

In [14]:
# Drop null rows
movie_us_df = movie_us_df.dropna(axis = 0,how= 'any')
movie_us_df.shape

(1480, 38)

In [15]:
# let's have a look at how many labels each variable has

for col in movie_us_df.columns:
    print(col, ': ', len(movie_us_df[movie_cat].nunique()), ' labels')

title :  18  labels
original_title :  18  labels
runtime :  18  labels
budget :  18  labels
revenue :  18  labels
popularity :  18  labels
vote_average :  18  labels
vote_count :  18  labels
genres :  18  labels
original_language :  18  labels
spoken_languages :  18  labels
country :  18  labels
production_companies :  18  labels
production_countries :  18  labels
distributor :  18  labels
producers :  18  labels
director :  18  labels
starring :  18  labels
cinematography :  18  labels
editors :  18  labels
writers :  18  labels
composers :  18  labels
rating_0.5 :  18  labels
rating_1.0 :  18  labels
rating_1.5 :  18  labels
rating_2.0 :  18  labels
rating_2.5 :  18  labels
rating_3.0 :  18  labels
rating_3.5 :  18  labels
rating_4.0 :  18  labels
rating_4.5 :  18  labels
rating_5.0 :  18  labels
year_film :  18  labels
year_ceremony :  18  labels
ceremony :  18  labels
category :  18  labels
name :  18  labels
winner :  18  labels


In [16]:
# let's examine how many columns we will obtain after one hot encoding these variables
encode_df=pd.get_dummies(movie_us_df[movie_cat], drop_first=True)
encode_df.shape

(1480, 5732)

In [17]:
# Create a title Dataframe.
title_df=pd.DataFrame(data=movie_us_df['title'],index=movie_us_df.index,copy=True)
title_df.head()

,title
6,Awakenings
7,Awakenings
8,Awakenings
9,Days of Thunder
10,Dick Tracy


In [18]:
# Merge one-hot encoded features and drop the originals
oscar_us_df = movie_us_df.merge(encode_df,left_index=True, right_index=True)
oscar_us_df = oscar_us_df.drop(movie_cat,1)
oscar_us_df.head(2)

,runtime,budget,revenue,popularity,vote_average,vote_count,rating_0.5,rating_1.0,rating_1.5,rating_2.0,...,name_Written by Tony Gilroy,name_Written by Wes Anderson & Owen Wilson,name_Written by Wes Anderson & Roman Coppola,name_Written for the Screen by Gary Ross,name_Written for the Screen by Paul Thomas Anderson,name_Written for the screen by Greta Gerwig,name_Written for the screen by Joel Coen & Ethan Coen,name_Written for the screen by Paul Thomas Anderson,name_Wylie Stateman,name_Wylie Stateman and Renée Tondelli
6,120.0,29000000.0,52096475.0,13.201595,7.6,568.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,120.0,29000000.0,52096475.0,13.201595,7.6,568.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


#### Classification (RandomForestClassification)

In [19]:
# Define the features set.
X = oscar_us_df.copy()
X = X.drop('winner', axis=1)
X.shape

(1480, 5751)

In [20]:
# Define the target set(output label)
y = oscar_us_df['winner']
y.head(2)

6    0
7    0
Name: winner, dtype: int64

In [21]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Creating a StandardScaler instance.
scaler = StandardScaler()

In [23]:
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [24]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create a random forest classifier.
#rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [26]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [27]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,

In [28]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,258,37
Actual 1,49,26


In [29]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.7675675675675676

In [30]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,258,37
Actual 1,49,26


Accuracy Score : 0.7675675675675676
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.87      0.86       295
           1       0.41      0.35      0.38        75

    accuracy                           0.77       370
   macro avg       0.63      0.61      0.62       370
weighted avg       0.75      0.77      0.76       370



In [31]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([6.64822096e-03, 5.62403118e-03, 7.79208860e-03, ...,
       8.92096866e-08, 0.00000000e+00, 2.40800840e-07])

In [32]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.015019849100019822, 'category_FILM EDITING'),
 (0.012737959012438635, 'category_BEST PICTURE'),
 (0.011183588364833243, 'category_ACTRESS IN A SUPPORTING ROLE'),
 (0.010430558277438901, 'category_CINEMATOGRAPHY'),
 (0.010060242333178177, 'category_DIRECTING'),
 (0.009222402211499914, 'category_ACTOR IN A SUPPORTING ROLE'),
 (0.009063978106908477, 'category_ACTRESS IN A LEADING ROLE'),
 (0.008799231629169506, 'category_MUSIC (Original Score)'),
 (0.008369182300854473, 'category_ACTOR IN A LEADING ROLE'),
 (0.00834215998442161, 'vote_average'),
 (0.007886799080549583, 'ceremony'),
 (0.007862755425867816, 'vote_count'),
 (0.007792088599017241, 'revenue'),
 (0.007772244213954256, 'year_ceremony'),
 (0.007457320391570997, 'category_MAKEUP'),
 (0.0071847216616609585, 'year_film'),
 (0.007068653381505436, 'category_SOUND'),
 (0.006665139614916701,
  'category_WRITING (Screenplay Written Directly for the Screen)'),
 (0.0066482209627295825, 'runtime'),
 (0.0064306082829218465, 'popularity')